<a href="https://colab.research.google.com/github/maralthesage/visualizing_cnns/blob/main/Thesis_Model_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Pipeline

In this notebook we train VGG16 - MobileNet - Inception V3 in 3 modes of:
1. Training from scratch
2. Used as a feature extractor with pretrained weights of the `imagenet` with a dense layer classifier on top
3. Fine-tuned with the weights of `imagenet` with a lr = 1e-5.

All three modes are done using a balanced subset of PlantVillage Dataset and is done for 40 epochs.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras import Model
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns


# Model Configurations

To train a model from (VGG16, MobileNet or Inception V3) please specify the `model_name` in the following way:

* VGG for VGG16
* MN for MobileNet
* IN for Inception V3

The `mode` will be important in the next phase where you need to choose the setting for training and the choices are:

* TR for training from scratch
* FX for feature extraction
* FT for Fine-tuning

These parameters determine the name of the saved models, which won't be required here.

In [ ]:
## choose the name of models from the following list ==> 
## ['VGG-FT-40','VGG-FX-40','VGG-TR-40', 'MN-FT-40','MN-FX-40','MN-TR-40', 'IN-FT-40','IN-FX-40','IN-TR-40']
path = '/content/drive/MyDrive/Colab Notebooks'
mode = 'TR'
model_name = 'MN'

## models' paths are relative and the following address needs to be changed.
models_path = f'{path}/{model_name}-{mode}'

model = tf.keras.models.load_model(models_path)

if 'VGG' in model_name:
  IMG_SIZE = 224
  preprocess_input = tf.keras.applications.vgg16.preprocess_input

elif 'MN' in model_name:
  IMG_SIZE = 224
  preprocess_input = tf.keras.applications.mobilenet.preprocess_input
  
elif 'IN' in model_name:
  IMG_SIZE = 299
  preprocess_input = tf.keras.applications.inception_v3.preprocess_input


# model.summary()


# Setting up Datasets

Below we use `ImageDataGenerator` method from keras to create our dataset as well as preprocess them in one step. `IMG_SIZE` come from the above block where we chose the model.

In [ ]:
BATCH_SIZE = 32

data_dir = "/content/drive/MyDrive/output/"


train_dataset = ImageDataGenerator(preprocessing_function=preprocess_input, zoom_range=0.2, rotation_range=20)

validation_dataset = ImageDataGenerator(preprocessing_function=preprocess_input)
test_dataset = ImageDataGenerator(preprocessing_function=preprocess_input)


# train_dataset = train_dataset.flow_from_directory(
#         data_dir+'train',
#         target_size= (IMG_SIZE, IMG_SIZE),
#         class_mode="categorical",
#         seed = 42,
#         batch_size=BATCH_SIZE,
#         shuffle=True)

# validation_dataset = validation_dataset.flow_from_directory(
#         data_dir+'valid',
#         target_size= (IMG_SIZE, IMG_SIZE),
#         class_mode="categorical",
#         seed = 42,
#         batch_size=BATCH_SIZE,
#         shuffle=True)

test_dataset = test_dataset.flow_from_directory(
        data_dir+'test',
        target_size= (IMG_SIZE, IMG_SIZE),
        class_mode="categorical",
        seed = 42,
        batch_size=BATCH_SIZE,
        shuffle=False)



## Creating the base model 

There are three sub blocks in the following code block, each are for TR, FX, and FT modes of training in that order. they are separated by two horizontal lines. 

* In case you want to train a model from scratch, keep the first subblock uncommented and the other two commented (as it is here).

* In case you want to use the base_model as a feature extractor, commented out the 1 and 3 subblocks and uncomment the 2nd subblock

* In case you want the base_model to be fine-tuned, comment out the 1st and 2nd subblocks and uncomment the third subblock.

In [ ]:
## Run the following Code for TR mode (Training from Scratch)
base_model = tf.keras.applications.vgg16.VGG16(weights = None ,include_top = False, input_shape= (IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = True

### ------------------------------------------------- ###
# ## Run the following Code for FX mode (Feature Extraction)

# base_model = tf.keras.applications.vgg16.VGG16(weights = 'imagenet' ,include_top = False, input_shape= (IMG_SIZE, IMG_SIZE, 3))
# base_model.trainable = False

### ------------------------------------------------- ###
# ## Run the following Code for FT mode (Fine Tuning)

# base_model = tf.keras.applications.vgg16.VGG16(weights = 'imagenet' ,include_top = False, input_shape= (IMG_SIZE, IMG_SIZE, 3))
# base_model.trainable = True

Next, you only need to uncomment one of the `opt` which stands for optimizers based on the training mode you are working in. in case of FT mode, uncomment the first `opt` and for TR and FT modes, uncomment the 2nd `opt` so that the learning rate is adjusted. 

You can see the `model.summary()` by uncommenting it.

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
out = Dense(38, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=out)

### ------------------------------------------------- ###
## Uncomment this in case you are in FT mode
# opt = tf.keras.optimizers.Adam(learning_rate = 1e-5)

## Uncomment this in case you are in TR or FX modes
# opt = tf.keras.optimizers.Adam(learning_rate = 1e-3)
### ------------------------------------------------- ###

model.compile(optimizer= opt, loss='categorical_crossentropy', metrics= 'accuracy')
# model.summary()

## Fitting the model

Here, nothing needs to be changed. The `%%time` function will show the training time at the end of the 40th epoch. However, it may only work in Google Colab (in case you are running this code elsewhere).
 

In [ ]:
%%time

N_EPOCHS = 40
history = model.fit(train_dataset, epochs = N_EPOCHS, validation_data = validation_dataset)

You won't need to run the following block because that was for saving the models which has been already done.

In [ ]:
## Saving model
# save_path = "/content/drive/MyDrive/Colab Notebooks/SomeFolderOfInterest/"
# model.save(save_path + model_name)

The following code is for visualizing the accuracy and loss progress during training. It only works when the model is trained in this runtime (and now loaded as a saved model). It plots teh training and validation accuracies and loss.

In [ ]:
# Visualize training accuracy and loss
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title(f'model accuracy for {model_name}')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(f'{save_path}{model_name}-acc.jpg')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title(f'model loss for {model_name}')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(f'{save_path}{model_name}-loss.jpg')
plt.show()


## Computing the Confusion Matrix

In order to better understand and visualize the resutls and how the model works, we also adopted ConfusionMatrix from Scikit-learn library and the following code, visualize that.

But first, we need to define class_names, and then determine `y_true` using the true labels of the data which is done in the following codeblocks.

In [ ]:
class_names = test_dataset.class_indices
num_classes = len(class_names)
class_names = list(class_names.keys())

In [ ]:
def true_label(filepath):
  for name in class_names:
    if name in filepath:
      return name

In [ ]:
# defining y_true

filenames = test_dataset.filenames
y_true = np.zeros(len(filenames))

for id, filename in enumerate(filenames):
  y_true[id] = class_names.index(true_label(filename))


In [ ]:
# compute y_pred
preds = model.predict(test_dataset)
y_pred = np.argmax(preds,axis=1)

In [ ]:
cm = confusion_matrix(y_true=y_true,y_pred=y_pred)

# cm_display = ConfusionMatrixDisplay(cm).plot()
plt.figure(figsize = (10,8))

# Create Confusion Matrix
b = sns.heatmap(cm, annot=True,cmap='viridis')

# Set the Title
b.set(title='Confusion Matrix')

# Set the Labels
b.set(xlabel='Predicted', ylabel='True')

plt.show()